In [ ]:
# !pip install protobuf tensorflow==2.11.0
# !pip uninstall tensorrt -y

In [ ]:
# !pip install tensorflow_addons -U

In [ ]:
%load_ext tensorboard

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib

from IPython import display

In [ ]:
import datetime

## File Parsing 

In [ ]:
d1 = '/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2/client'
d2 = '/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2/counselor'
# iterate over files in
# that directory

data_in = []
data_out = []
for filename in sorted(os.listdir(d1)):
    fn = os.path.join(d1, filename)
    if os.path.isfile(fn):
        with open(fn) as f:
            lines = f.readlines()
            aline = []
            for x in lines:
                # print(x.rstrip('\n').split(" ")[1:5])
                x_low = x.rstrip('\n').split(" ")[1]
                x_low = int(x_low[2:len(x_low)-1])
                y_low = x.rstrip('\n').split(" ")[2]
                y_low = int(y_low[0:len(y_low)-1])
                x_high = x.rstrip('\n').split(" ")[3]
                x_high = int(x_high[1:len(x_high)-1])
                y_high = x.rstrip('\n').split(" ")[4]
                y_high = int(y_high[0:len(y_high)-2])
                norm_x = (int(x.rstrip('\n').split(" ")[-2])-x_low)/(x_high-x_low)
                norm_y = (int(x.rstrip('\n').split(" ")[-1])-y_low)/(y_high-y_low)
                aline.append([norm_x, norm_x])
            data_in.append(aline)
        f.close()


for filename in sorted(os.listdir(d2)):
    fn = os.path.join(d2, filename)
    # checking if it is a file
    if os.path.isfile(fn):
        with open(fn) as f:
            lines = f.readlines()
            aline = []
            for x in lines:
                x_low = x.rstrip('\n').split(" ")[1]
                x_low = int(x_low[2:len(x_low)-1])
                y_low = x.rstrip('\n').split(" ")[2]
                y_low = int(y_low[0:len(y_low)-1])
                x_high = x.rstrip('\n').split(" ")[3]
                x_high = int(x_high[1:len(x_high)-1])
                y_high = x.rstrip('\n').split(" ")[4]
                y_high = int(y_high[0:len(y_high)-2])
                norm_x = (int(x.rstrip('\n').split(" ")[-2])-x_low)/(x_high-x_low)
                norm_y = (int(x.rstrip('\n').split(" ")[-1])-y_low)/(y_high-y_low)
                aline.append([norm_x, norm_x])
            data_out.append(aline)
        f.close()
        

In [ ]:
# data_in = data_in[:len(data_out)]
# # print(tf.constant(data_in).shape)
# # print(tf.constant(data_out).shape)

# data = []
# sequence_len = 16
# ds = tf.data.Dataset.from_tensor_slices((tf.constant(data_in), tf.constant(data_out)))
# ds = ds.batch(sequence_len)
# ds = ds.shuffle(ds.cardinality().numpy())
# for inn, outt in ds:
#     print(inn.shape, outt.shape)

## U-Net Generator

In [ ]:
class GU(Model):

    def __init__(self, in_dim, num_channels=2, emb_dim=512, gate_filters=32, num_resolutions=3, attn_res_idx=1):
        super().__init__()
        
        self.in_dim = in_dim
        self.num_channels = num_channels
        self.num_resolutions = num_resolutions
        self.attn_res_idx = attn_res_idx
        self.emb_dim = emb_dim
        self.gate_filters = gate_filters
#         self.model = self.build_graph()
        
#     def timestep_embedding(self, ts, embedding_dim):
#         assert len(ts.shape)==1
        
#         half_dim = embedding_dim // 2
#         emb = -(tf.math.log(10000.0)/(half_dim-1))
        
#         idxs = tf.range(half_dim, dtype=tf.float32)
#         emb = tf.math.exp(idxs*emb)
        
#         ts = tf.cast(ts, dtype=tf.float32)
        
#         emb = ts[:,None]* emb[None,:]
        
#         pe = tf.concat([tf.sin(emb), tf.cos(emb)], axis=1)
        
#         return pe
    
    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)
        
    def ResBlK(self, x, temb=None):
        h = self.nonlinearity(self.normalize(x))
        h = layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(h)
        if not temb == None:
            temb = self.nonlinearity(temb)
            h +=  tf.reshape(layers.Dense(h.shape[-1])(temb), (-1, 1, 1,h.shape[-1]))

        h = self.nonlinearity(self.normalize(h))
        h = layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(h)
        
        return x+h
        
    def downsample(self, x, filters, factor=2):
        return layers.Conv2D(filters, (3, 3),
                                        strides=(factor, factor), padding='same')(x)

    def upsample(self, x, filters, factor=2):
        return layers.Conv2DTranspose(filters, (3, 3),
                                       strides=(factor, factor), padding='same')(x)
        
    def NonlocalGaussian(self, x):
        h = self.normalize(x)
        W_QK = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')
        Q = W_QK(x)
        K = W_QK(x)
        
        srdK = tf.math.sqrt(tf.cast(K.shape[1]*K.shape[2]*K.shape[3], tf.float32))
        V = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        # phi = tf.transpose(phi, (0, 3, 2, 1))

        Q = tf.reshape(Q, shape=(-1, Q.shape[1]*Q.shape[2], Q.shape[3]))

        K = tf.reshape(K, shape=(-1, K.shape[1]*K.shape[2], K.shape[3]))
        V = tf.reshape(V, shape=(-1, V.shape[1]*V.shape[2], V.shape[3]))
        
        rnQ = tf.reduce_sum(tf.math.square(Q), axis=-1, keepdims=True)
        f1 = tf.matmul(rnQ, tf.ones_like(rnQ), transpose_b=True)

        f2 = tf.matmul(Q, K, transpose_b=True)

        rnK = tf.reduce_sum(tf.math.square(K), axis=-1, keepdims=True)
        f3 = tf.matmul(tf.ones_like(rnK), rnK, transpose_b=True)

        f = (f1-2*f2+f3)/srdK
        
        f = layers.Softmax()(-f)
        
        y = tf.matmul(f, V)
        
        y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
        z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        return z
    
    def call(self, img_input):
        
        hs = [layers.Conv2D(self.gate_filters, (1, 1),
                                        strides=(1, 1), padding='same')(img_input)]
        
        for i in range(self.num_resolutions):
            h = self.ResBlK(hs[-1])
            if i == self.attn_res_idx:
                h = self.NonlocalGaussian(h)
                
            if i != self.num_resolutions - 1:
                h = self.downsample(h, h.shape[-1]*2)
                
            hs.append(h)

        h = hs[-1]
        h = self.ResBlK(h)
        h = self.NonlocalGaussian(h)
        h = self.ResBlK(h)
                
        for i in reversed(range(self.num_resolutions)):
            if i == self.attn_res_idx:
                h = self.NonlocalGaussian(h)   
            h = self.ResBlK(h)
            h = h+hs[i]
                
            if i != 0:
                h = self.upsample(h, h.shape[-1]//2)
                
        h = self.nonlinearity(self.normalize(h))
        
        h = layers.Conv2D(self.num_channels, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        
        return h
    
#     def call(self, x):
#         return self.model([x])
    
    def build_model(self):
        x = layers.Input(shape=(self.in_dim[0],
                                        self.in_dim[1], self.num_channels))
        
        return Model(inputs=[x], outputs=self.call(x))
    
    def build_graph(self):
        self.build(input_shape=(None,self.in_dim[0],
                                        self.in_dim[1], self.num_channels))
        

In [ ]:
unet1=GU((16, 68))
# unet1.build_graph()

In [ ]:
model1 = unet1.build_model()

In [ ]:
model1.summary()

In [ ]:
tf.keras.utils.plot_model(model1,to_file="./imgs/model.png", show_shapes=True, show_layer_activations=True)

In [ ]:
# tf.keras.utils.model_to_dot(model1, show_shapes=True, show_layer_activations=True).write_raw("model.dot")
# graph.write_raw("output_raw.dot")

## Supervised Model

In [ ]:
EPSILON = 1e-16

class model:
    def __init__(self, dataset_path, data_shape, num_channels, train_split=0.8,
                 batch_size=64, lr=3e-4):
        self.s = data_shape
        self.nc = num_channels
        self.bs = batch_size
        # self.dum = dum
        
        if not dataset_path==None:
            cl_path = dataset_path + "/client"
            co_path = dataset_path + "/counselor"

            self.ds = self.make_dataset(cl_path, co_path)
            
            ds_size = self.ds.cardinality().numpy()
            train_size = int(train_split * ds_size)
            # test_size = int((1.0-train_split) * ds_size)

            self.train_ds = self.ds.take(train_size)    
            self.test_ds = self.ds.skip(train_size)
        else:
            print("WARNING: Dataset not loaded, Model in Generator mode")
        
        self.g = GU(self.s, self.nc).build_model()
        
        ### Optimizer setups ###
        
        self.g_opt = tf.keras.optimizers.Adam(lr,beta_1=0., weight_decay=0.01)
        
        ### Optimizer setups ###

    def make_dataset(self, d1, d2):
        data_in = []
        data_out = []
        for filename in sorted(os.listdir(d1)):
            fn = os.path.join(d1, filename)
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        # print(x.rstrip('\n').split(" ")[1:5])
                        x_low = x.rstrip('\n').split(" ")[1]
                        x_low = int(x_low[2:len(x_low)-1])
                        y_low = x.rstrip('\n').split(" ")[2]
                        y_low = int(y_low[0:len(y_low)-1])
                        x_high = x.rstrip('\n').split(" ")[3]
                        x_high = int(x_high[1:len(x_high)-1])
                        y_high = x.rstrip('\n').split(" ")[4]
                        y_high = int(y_high[0:len(y_high)-2])
                        norm_x = (int(x.rstrip('\n').split(" ")[-2])-x_low)/(x_high-x_low)
                        norm_y = (int(x.rstrip('\n').split(" ")[-1])-y_low)/(y_high-y_low)
                        aline.append([norm_x, norm_x])
                    data_in.append(aline)
                f.close()

        for filename in sorted(os.listdir(d2)):
            fn = os.path.join(d2, filename)
            # checking if it is a file
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        x_low = x.rstrip('\n').split(" ")[1]
                        x_low = int(x_low[2:len(x_low)-1])
                        y_low = x.rstrip('\n').split(" ")[2]
                        y_low = int(y_low[0:len(y_low)-1])
                        x_high = x.rstrip('\n').split(" ")[3]
                        x_high = int(x_high[1:len(x_high)-1])
                        y_high = x.rstrip('\n').split(" ")[4]
                        y_high = int(y_high[0:len(y_high)-2])
                        norm_x = (int(x.rstrip('\n').split(" ")[-2])-x_low)/(x_high-x_low)
                        norm_y = (int(x.rstrip('\n').split(" ")[-1])-y_low)/(y_high-y_low)
                        aline.append([norm_x, norm_x])
                    data_out.append(aline)
                f.close()
        data_in = data_in[:len(data_out)]
        # data = []
        sequence_len = self.s[0]
        ds = tf.data.Dataset.from_tensor_slices((tf.constant(data_in), tf.constant(data_out)))
        ds = ds.batch(sequence_len)
        ds = ds.shuffle(ds.cardinality().numpy())
        
        return ds
    
    def loss_func(self, pred_co, gt_co):
        return tf.nn.compute_average_loss(tf.reduce_sum(tf.math.add(pred_co, -tf.cast(gt_co, tf.float32))**2, axis = [1,2,3]))
        
    @tf.function
    def update(self, cl, co):
        with tf.GradientTape() as g_tape:
            pred_co = self.g(cl)
            g_loss = self.loss_func(pred_co, co)
            
        grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
        self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
        
        return g_loss
        
    def train(self, epochs=50):
        for epo in range(epochs):
            g_losses = []
            
            batch_inn = []
            batch_outt = []

            for inn, outt in self.train_ds:
                if inn.shape == (8,68,2) and outt.shape == (8,68,2):
                    if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                        tf_batch_inn = tf.cast(tf.stack(batch_inn), dtype=tf.float32)
                        tf_batch_outt = tf.cast(tf.stack(batch_outt), dtype=tf.float32)
                        g_losses.append(self.update(tf_batch_inn, tf_batch_outt))
                        batch_inn = []
                        batch_outt = []
                    else:
                        batch_inn.append(inn)
                        batch_outt.append(outt)


            print("Epoch {:04d}".format(epo), "Generator Avg. Loss: ", np.mean(g_losses), flush=True)
    def test(self):
        errors = []
        batch_inn = []
        batch_outt = []
        for inn, outt in self.test_ds:
            if inn.shape == (8,68,2) and outt.shape == (8,68,2):
                if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                    tf_batch_inn = tf.stack(batch_inn)
                    tf_batch_outt = tf.stack(batch_outt)
                    pred_co = self.g(tf_batch_inn)
                    errors.append(self.loss_func(pred_co, tf_batch_outt))
                    batch_inn = []
                    batch_outt = []
                else:
                    batch_inn.append(inn)
                    batch_outt.append(outt)
        print("Test Set Avg. Loss: ", np.mean(errors), flush=True)
        

In [ ]:
ds_path = "/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2"

In [ ]:
m1 = model(ds_path, (8,68), 2)

In [ ]:
m1.train(50)

In [ ]:
m1.test()

## Attention Discriminator

In [ ]:
class AttnD(Model):
    
    def __init__(self, in_dim ,num_out_filter=1,
                  gate_filters=32):
#                  stage_filters=(256,256,256,256,256,256,256), stage_kernels=(3,3,3,3,3,3,3), stage_strides_ds=(2,2,2,2,2,2)):
        
        super().__init__()
        # staging params must be equal
        # assert len(stage_filters) == len(stage_kernels)
        # assert len(stage_filters) == len(stage_strides_ds)+1
        
        self.gate_filters = gate_filters
        self.in_dim = in_dim
#         self.attn_shape = attn_shape
        self.num_out_filter = num_out_filter

    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)
    
    def downsample(self, x, filters, factor=2):
        return layers.Conv2D(filters, (3, 3),
                                        strides=(factor, factor), padding='same')(x)
    
    def NonlocalGaussian(self, x):
        h = x
        
        theta = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        srdphi = tf.math.sqrt(tf.cast(phi.shape[1]*phi.shape[2]*phi.shape[3], tf.float32))
        
        g = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        phi = tf.transpose(phi, (0, 3, 2, 1))

        theta = tf.reshape(theta, shape=(-1, theta.shape[1]*theta.shape[2], theta.shape[3]))

        phi = tf.reshape(phi, shape=(-1, phi.shape[1], phi.shape[2]*phi.shape[3]))
        g = tf.reshape(g, shape=(-1, g.shape[1]*g.shape[2], g.shape[3]))
        
        f = tf.matmul(theta, phi)/srdphi
        
        f = layers.Softmax()(f)
        
        y = tf.matmul(f, g)
        
        y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
        z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        z = self.normalize(z)
        
        return z

    def call(self, latent):
        
#         h = layers.Dense(self.attn_shape[0]*self.attn_shape[1]*self.gate_filters)(latent)
#         h = layers.Reshape((self.attn_shape[0], self.attn_shape[1], self.gate_filters))(h)
        h = layers.Conv2D(self.gate_filters, (1, 1),
                                        strides=(1, 1), padding='same')(latent)
        h = self.NonlocalGaussian(h)
        h=self.downsample(h,int(self.gate_filters*2.))
        h = self.NonlocalGaussian(h)
        h=self.downsample(h,int(self.gate_filters*2.))
        h = layers.Flatten()(h)
        h = layers.Dense(self.num_out_filter)(h)
        h = self.nonlinearity(h)
    
        return h

    def build_model(self):
        x = layers.Input(shape=self.in_dim)
        
        return Model(inputs=[x], outputs=self.call(x))

    
    def build_graph(self):
        self.build(input_shape=(None,self.in_dim[0],
                                        self.in_dim[1], self.in_dim[2]))
        
    # def reproduce_model(self):
    #     x = layers.Input(shape=(self.image_shape[0],
    #                                     self.image_shape[1], self.num_channel))
        
    #     return Model(inputs=x, outputs=self.call(x))
    
#     def plot_models(self,path):
#         rmodel = self.reproduce_model()
#         tf.keras.utils.plot_model(rmodel, to_file=path+"/model.png", show_shapes=True, show_layer_activations=True)
        
#         idx = 0
#         for resblk in self.resblks:
#             tf.keras.utils.plot_model(resblk.reproduce_model(self.res_inshape[idx], (1,1)), to_file=path+"/resblk_{:04d}.png".format(idx), show_shapes=True, show_layer_activations=True)
#             idx += 1


## L2 Attention Discriminator

In [ ]:
class L2AttnD(Model):
    
    def __init__(self, in_dim ,num_out_filter=1,
                  gate_filters=32):
#                  stage_filters=(256,256,256,256,256,256,256), stage_kernels=(3,3,3,3,3,3,3), stage_strides_ds=(2,2,2,2,2,2)):
        
        super().__init__()
        # staging params must be equal
        # assert len(stage_filters) == len(stage_kernels)
        # assert len(stage_filters) == len(stage_strides_ds)+1
        
        self.gate_filters = gate_filters
        self.in_dim = in_dim
#         self.attn_shape = attn_shape
        self.num_out_filter = num_out_filter

    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)
    
    def downsample(self, x, filters, factor=2):
        return layers.Conv2D(filters, (3, 3),
                                        strides=(factor, factor), padding='same')(x)
    
    def NonlocalGaussian(self, x_in, y_in):
        hx = x_in
        hy = y_in
        W_QK = layers.Conv2D(hx.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')
        Q = W_QK(hx)
        K = W_QK(hy)
        
        srdK = tf.math.sqrt(tf.cast(K.shape[1]*K.shape[2]*K.shape[3], tf.float32))
        V = layers.Conv2D(hx.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(hy)
        # phi = tf.transpose(phi, (0, 3, 2, 1))

        Q = tf.reshape(Q, shape=(-1, Q.shape[1]*Q.shape[2], Q.shape[3]))

        K = tf.reshape(K, shape=(-1, K.shape[1]*K.shape[2], K.shape[3]))
        V = tf.reshape(V, shape=(-1, V.shape[1]*V.shape[2], V.shape[3]))
        
        rnQ = tf.reduce_sum(tf.math.square(Q), axis=-1, keepdims=True)
        f1 = tf.matmul(rnQ, tf.ones_like(rnQ), transpose_b=True)

        f2 = tf.matmul(Q, K, transpose_b=True)

        rnK = tf.reduce_sum(tf.math.square(K), axis=-1, keepdims=True)
        f3 = tf.matmul(tf.ones_like(rnK), rnK, transpose_b=True)

        f = (f1-2*f2+f3)/srdK
        
        f = layers.Softmax()(-f)
        
        y = tf.matmul(f, V)
        
        y = tf.reshape(y, (-1, hx.shape[1], hx.shape[2], y.shape[-1]))
        
        z = tf.math.add(x_in, layers.Conv2D(hx.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        z = self.normalize(z)
        
        return z

    def call(self, xy):
        
        x, y = xy
#         h = layers.Dense(self.attn_shape[0]*self.attn_shape[1]*self.gate_filters)(latent)
#         h = layers.Reshape((self.attn_shape[0], self.attn_shape[1], self.gate_filters))(h)
        hx = layers.Conv2D(self.gate_filters, (1, 1),
                                        strides=(1, 1), padding='same')(x)
        hy = layers.Conv2D(self.gate_filters, (1, 1),
                                            strides=(1, 1), padding='same')(y)
        hx = self.NonlocalGaussian(hx, hx)
        hy = self.NonlocalGaussian(hy, hy)
        hx = self.downsample(hx,int(self.gate_filters*2.))
        hy = self.downsample(hy,int(self.gate_filters*2.))
        h = self.NonlocalGaussian(hx, hy)
        h = self.downsample(h,int(self.gate_filters*2.))
        h = layers.Flatten()(h)
        h = layers.Dense(self.num_out_filter)(h)
        h = self.nonlinearity(h)
    
        return h

    def build_model(self):
        x = layers.Input(shape=self.in_dim)
        y = layers.Input(shape=self.in_dim)
        
        return Model(inputs=[x,y], outputs=self.call([x,y]))

    
    def build_graph(self):
        self.build(input_shape=[(None,self.in_dim[0],
                                        self.in_dim[1], self.in_dim[2]),(None,self.in_dim[0],
                                        self.in_dim[1], self.in_dim[2])])
        
    # def reproduce_model(self):
    #     x = layers.Input(shape=(self.image_shape[0],
    #                                     self.image_shape[1], self.num_channel))
        
    #     return Model(inputs=x, outputs=self.call(x))
    
#     def plot_models(self,path):
#         rmodel = self.reproduce_model()
#         tf.keras.utils.plot_model(rmodel, to_file=path+"/model.png", show_shapes=True, show_layer_activations=True)
        
#         idx = 0
#         for resblk in self.resblks:
#             tf.keras.utils.plot_model(resblk.reproduce_model(self.res_inshape[idx], (1,1)), to_file=path+"/resblk_{:04d}.png".format(idx), show_shapes=True, show_layer_activations=True)
#             idx += 1


In [ ]:
d = L2AttnD([16,68,2])

In [ ]:
d1 = d.build_model()

In [ ]:
tf.keras.utils.plot_model(d1, to_file="./model_plots/model.png", show_shapes=True, show_layer_activations=True)

In [ ]:
# d1.summary()

## GAN Model

In [ ]:
EPSILON = 1e-16

class gan_model:
    def __init__(self, dataset_path, data_shape, num_channels, dum=5, train_split=0.8,
                 batch_size=64, lr=3e-4, gp_lam=10.0):
        self.s = data_shape
        self.nc = num_channels
        self.bs = batch_size
        self.gp_lam = gp_lam
        self.dum = dum
        self.in_shape = list(self.s)
        self.in_shape.append(self.nc)
        
        if not dataset_path==None:
            cl_path = dataset_path + "/client"
            co_path = dataset_path + "/counselor"

            self.ds = self.make_dataset(cl_path, co_path)
            
            ds_size = self.ds.cardinality().numpy()
            train_size = int(train_split * ds_size)
            # test_size = int((1.0-train_split) * ds_size)

            self.train_ds = self.ds.take(train_size)    
            self.test_ds = self.ds.skip(train_size)
        else:
            print("WARNING: Dataset not loaded, Model in Generator mode")
        
        self.g = GU(self.s, self.nc).build_model()
        self.d = L2AttnD(self.in_shape).build_model()

        
        ### Optimizer setups ###
        
        self.g_opt = tf.keras.optimizers.Adam(lr,beta_1=0., weight_decay=0.01, global_clipnorm=True)
        self.d_opt = tf.keras.optimizers.Adam(lr,beta_1=0., weight_decay=0.01, global_clipnorm=True)
        
        ### Optimizer setups ###


    def make_dataset(self, d1, d2):
        data_in = []
        data_out = []
        for filename in sorted(os.listdir(d1)):
            fn = os.path.join(d1, filename)
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        # print(x.rstrip('\n').split(" ")[1:5])
                        x_low = x.rstrip('\n').split(" ")[1]
                        x_low = int(x_low[2:len(x_low)-1])
                        y_low = x.rstrip('\n').split(" ")[2]
                        y_low = int(y_low[0:len(y_low)-1])
                        x_high = x.rstrip('\n').split(" ")[3]
                        x_high = int(x_high[1:len(x_high)-1])
                        y_high = x.rstrip('\n').split(" ")[4]
                        y_high = int(y_high[0:len(y_high)-2])
                        norm_x = (int(x.rstrip('\n').split(" ")[-2])-x_low)/(x_high-x_low)
                        norm_y = (int(x.rstrip('\n').split(" ")[-1])-y_low)/(y_high-y_low)
                        aline.append([norm_x, norm_x])
                    data_in.append(aline)
                f.close()

        for filename in sorted(os.listdir(d2)):
            fn = os.path.join(d2, filename)
            # checking if it is a file
            if os.path.isfile(fn):
                with open(fn) as f:
                    lines = f.readlines()
                    aline = []
                    for x in lines:
                        x_low = x.rstrip('\n').split(" ")[1]
                        x_low = int(x_low[2:len(x_low)-1])
                        y_low = x.rstrip('\n').split(" ")[2]
                        y_low = int(y_low[0:len(y_low)-1])
                        x_high = x.rstrip('\n').split(" ")[3]
                        x_high = int(x_high[1:len(x_high)-1])
                        y_high = x.rstrip('\n').split(" ")[4]
                        y_high = int(y_high[0:len(y_high)-2])
                        norm_x = (int(x.rstrip('\n').split(" ")[-2])-x_low)/(x_high-x_low)
                        norm_y = (int(x.rstrip('\n').split(" ")[-1])-y_low)/(y_high-y_low)
                        aline.append([norm_x, norm_x])
                    data_out.append(aline)
                f.close()

        data_in = data_in[:len(data_out)]
        # data = []
        sequence_len = self.s[0]
        ds = tf.data.Dataset.from_tensor_slices((tf.constant(data_in), tf.constant(data_out)))
        ds = ds.batch(sequence_len)
        ds = ds.shuffle(ds.cardinality().numpy())
        
        return ds
    
    def W_loss(self, d_data, d_gen, cl, x_it):
        
        with tf.GradientTape() as t_gp:
            t_gp.watch(x_it)
            d_it = self.d([cl,x_it])

        gp_grad = t_gp.gradient(d_it, x_it)
        l2n_gp = tf.math.sqrt(tf.reduce_sum(gp_grad**2, axis = [1,2,3])+EPSILON)
        L_gp = tf.expand_dims(self.gp_lam*(l2n_gp-1.0)**2, -1)

        L_g = d_data - d_gen

        L_d = -L_g + L_gp
        
        return tf.nn.compute_average_loss(L_g), tf.nn.compute_average_loss(L_d)
    
    def loss_func(self, pred_co, gt_co):
        print(tf.reduce_mean(pred_co), tf.reduce_mean(gt_co))
        return tf.reduce_mean(tf.reduce_sum(tf.math.add(pred_co, -tf.cast(gt_co, tf.float32))**2, axis = [1,2,3]))
        
    # @tf.function
    # def update(self, cl, co):
    #     with tf.GradientTape() as g_tape:
    #         pred_co = self.g(cl)
    #         g_loss = self.loss_func(pred_co, co)
            
    #     grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
    #     self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
        
    #     return g_loss
    
    @tf.function
    def update(self, cl, co, update_gen=True):
        
        # noise_input = tf.random.normal((imgs.shape[0], self.latent_dim))
            
        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
            gco = self.g(cl)

            dx = self.d([cl,co])
            dg = self.d([cl,gco])

            epsi = tf.random.uniform([co.shape[0], 1, 1, 1], 0.0, 1.0)
            co_it = tf.math.add(epsi*tf.cast(co,tf.float32), (1.0-epsi)*gco)

            g_loss, d_loss = self.W_loss(dx, dg, cl, co_it)

        if update_gen:
            grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
            grad_d = d_tape.gradient(d_loss, self.d.trainable_variables)

            self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
            self.d_opt.apply_gradients(zip(grad_d, self.d.trainable_variables))
        else:
            grad_d = d_tape.gradient(d_loss, self.d.trainable_variables)
            self.d_opt.apply_gradients(zip(grad_d, self.d.trainable_variables))

        return g_loss, d_loss

    def train(self, epochs=50):
        num_training = 0
        for epo in range(epochs):
            g_losses = []
            d_losses = []
            
            batch_inn = []
            batch_outt = []

            for inn, outt in self.train_ds:
                if list(inn.shape) == self.in_shape and list(outt.shape) == self.in_shape:
                    if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                        tf_batch_inn = tf.stack(batch_inn)
                        tf_batch_outt = tf.stack(batch_outt)
                        if num_training%self.dum == 0:
                            g_l, d_l = self.update(tf_batch_inn, tf_batch_outt, True)
                            g_losses.append(g_l.numpy())
                        else:
                            g_l, d_l = self.update(tf_batch_inn, tf_batch_outt, False)
                        d_losses.append(d_l.numpy())

                        num_training = (num_training+1)%self.dum
                        
                        batch_inn = []
                        batch_outt = []
                    else:
                        batch_inn.append(inn)
                        batch_outt.append(outt)


            print("Epoch {:04d}".format(epo), " Generator Avg. Loss: ", np.mean(g_losses),
                   ", Discriminator Avg. Loss: ", np.mean(d_losses), flush=True)
            
    def test(self):
        errors = []
        batch_inn = []
        batch_outt = []
        for inn, outt in self.test_ds:
            if inn.shape == (8,68,2) and outt.shape == (8,68,2):
                if (len(batch_inn))%self.bs ==0 and not len(batch_inn) == 0:
                    tf_batch_inn = tf.stack(batch_inn)
                    tf_batch_outt = tf.stack(batch_outt)
                    pred_co = self.g(tf_batch_inn)
                    errors.append(self.loss_func(pred_co, tf_batch_outt))
                    batch_inn = []
                    batch_outt = []
                else:
                    batch_inn.append(inn)
                    batch_outt.append(outt)
        print(errors)
        print("Test Set Avg. Loss: ", np.mean(errors), flush=True)
        

In [ ]:
ds_path = "/home/data_backup/data_bu/eva_dataset/materialForEEVA/images/session2"

In [ ]:
# gan = gan_model(ds_path, (16,68), 2)

In [ ]:
gan.train(epochs=10)

In [ ]:
# gan.test()

In [ ]:
# class L2SAttn(layers.Layer):
#     def __init__(self):
#         super().__init__()

#     def nonlinearity(self, x):
#         return tf.keras.activations.swish(x)
    
#     def normalize(self, x):
#         return tfa.layers.InstanceNormalization(axis=-1)(x)

#     def call(self, x):
#         h = self.normalize(x)
#         theta = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
#                                         strides=(1, 1), padding='same')(h)
#         phi = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
#                                         strides=(1, 1), padding='same')(h)
#         g = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
#                                         strides=(1, 1), padding='same')(h)
#         phi = tf.transpose(phi, (0, 3, 2, 1))

#         theta = tf.reshape(theta, shape=(-1, theta.shape[1]*theta.shape[2], theta.shape[3]))

#         phi = tf.reshape(phi, shape=(-1, phi.shape[1], phi.shape[2]*phi.shape[3]))
#         g = tf.reshape(g, shape=(-1, g.shape[1]*g.shape[2], g.shape[3]))
        
#         f = tf.matmul(theta, phi)
        
#         f = layers.Softmax()(f)
        
#         y = tf.matmul(f, g)
        
#         y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
#         z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
#                                         strides=(1, 1), padding='same')(y))
#         return z
    

In [ ]:
class L2SAttn_M(Model):

    def __init__(self, in_dim, num_channels=2):
        super().__init__()
        # self.l2sattn1 = L2SAttn()
        self.in_dim = in_dim
        self.nc = num_channels
        self.s = list(in_dim)
        self.s.append(self.nc)

    def nonlinearity(self, x):
        return tf.keras.activations.swish(x)
    
    def normalize(self, x):
        return tfa.layers.InstanceNormalization(axis=-1)(x)
    
    def call(self, x):
        h = self.normalize(x)
        Q = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        K = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        srdK = tf.math.sqrt(tf.cast(K.shape[1]*K.shape[2]*K.shape[3], tf.float32))
        V = layers.Conv2D(x.shape[-1]/2.0, (1, 1),
                                        strides=(1, 1), padding='same')(h)
        # phi = tf.transpose(phi, (0, 3, 2, 1))

        Q = tf.reshape(Q, shape=(-1, Q.shape[1]*Q.shape[2], Q.shape[3]))

        K = tf.reshape(K, shape=(-1, K.shape[1]*K.shape[2], K.shape[3]))
        V = tf.reshape(V, shape=(-1, V.shape[1]*V.shape[2], V.shape[3]))
        
        rnQ = tf.reduce_sum(tf.math.square(Q), axis=-1, keepdims=True)
        f1 = tf.matmul(rnQ, tf.ones_like(rnQ), transpose_b=True)

        f2 = tf.matmul(Q, K, transpose_b=True)

        rnK = tf.reduce_sum(tf.math.square(K), axis=-1, keepdims=True)
        f3 = tf.matmul(tf.ones_like(rnK), rnK, transpose_b=True)

        f = (f1-2*f2+f3)/srdK
        
        f = layers.Softmax()(-f)
        
        y = tf.matmul(f, V)
        
        y = tf.reshape(y, (-1, x.shape[1], x.shape[2], y.shape[-1]))
        
        z = tf.math.add(x, layers.Conv2D(x.shape[-1], (1, 1),
                                        strides=(1, 1), padding='same')(y))
        return z
    
    def build_graph(self):
        self.build(input_shape=(None,self.in_dim[0],
                                        self.in_dim[1], self.nc))

    def build_model(self):
        x = layers.Input(shape=self.s)
        
        return Model(inputs=[x], outputs=self.call(x))

In [ ]:
a = L2SAttn_M((32,32), 32)

In [ ]:
a.build_graph()

In [ ]:
# tf.keras.utils.plot_model(a.build_model(),to_file="./model_plots/model.png", show_shapes=True, show_layer_activations=True)

In [ ]:
Q = tf.random.uniform(shape=(2, 64, 32))
K = tf.random.uniform(shape=(2, 64, 32))

rnQ = tf.reduce_sum(tf.math.square(Q), axis=-1, keepdims=True)
f1 = tf.matmul(rnQ, tf.ones_like(rnQ), transpose_b=True)

f2 = tf.matmul(Q, K, transpose_b=True)

rnK = tf.reduce_sum(tf.math.square(K), axis=-1, keepdims=True)
f3 = tf.matmul(tf.ones_like(rnK), rnK, transpose_b=True)

srdK = tf.math.sqrt(tf.cast(K.shape[1]*K.shape[2], tf.float32))
# srdK
# f1, f2, f3

In [ ]:
# tf.ones(shape=(None,3))

## AffectNet

In [ ]:
train_valence_path="/home/data_backup/Downloads/affectnet_8labels/train_set/valence_sorted"
train_arousal_path="/home/data_backup/Downloads/affectnet_8labels/train_set/arousal_sorted"
train_landmark_path="/home/data_backup/Downloads/affectnet_8labels/train_set/landmarks_sorted"

In [ ]:
vals = []
for filename in sorted(os.listdir(train_valence_path)):
    fn = os.path.join(train_valence_path, filename)
    if os.path.isfile(fn):
        val = np.load(fn)
        vals.append(float(val))
        
aros = []
for filename in sorted(os.listdir(train_arousal_path)):
    fn = os.path.join(train_arousal_path, filename)
    if os.path.isfile(fn):
        aro = np.load(fn)
        aros.append(float(aro))

valaros = list(zip(vals, aros))

lnds = []
for filename in sorted(os.listdir(train_landmark_path)):
    fn = os.path.join(train_landmark_path, filename)
    if os.path.isfile(fn):
        lnd = np.load(fn)
        clnd = []
        for i in range(len(lnd)):
            clnd.append(float(lnd[i])/224.)
        lnds.append(clnd)


In [ ]:
cls_emo_map = {0:'happy', 1:'angry', 2:'sad', 3:'calm'}

In [ ]:
cls = []
for val, aro in valaros:
    if val >= 0 and aro >= 0:
        cls.append([1,0,0,0])
    if val < 0 and aro >= 0:
        cls.append([0,1,0,0])
    if val >= 0 and aro < 0:
        cls.append([0,0,0,1])
    if val < 0 and aro < 0:
        cls.append([0,0,1,0])

In [ ]:
len(cls), len(valaros)

In [ ]:
batch_size=64
ds = tf.data.Dataset.from_tensor_slices((lnds, cls))
ds = ds.shuffle(ds.cardinality().numpy())
ds = ds.batch(batch_size)
ds = ds.repeat(10) 

In [ ]:
# datapoint_test = None
for x,y in ds.take(1):
    print(x.shape)
    print(y)

In [ ]:
# f = tf.matmul(tf.expand_dims(datapoint_test, axis=-1), tf.expand_dims(datapoint_test, axis=-1), transpose_b=True)

# y = tf.squeeze(tf.matmul(f, tf.expand_dims(datapoint_test, axis=-1)))
# y.shape

### Architecture Modules

In [ ]:
from tensorflow import keras

In [ ]:
class ReflectionPadding2D(layers.Layer):
    """Implements Reflection Padding as a layer.

    Args:
        padding(tuple): Amount of padding for the
        spatial dimensions.

    Returns:
        A padded tensor with the same type as the input tensor.
    """

    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super().__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")

def NonlocalGaussian_1D(x_in):
    
    hx = x_in
    
    W_Q = layers.Dense(hx.shape[-1]/2.0)
    W_K = layers.Dense(hx.shape[-1]/2.0)
    
    Q = W_Q(hx)
    K = W_K(hx)

    srdK = tf.math.sqrt(tf.cast(K.shape[-1], tf.float32))
    
    V = layers.Dense(hx.shape[-1]/2.0)(hx)

    f = tf.matmul(tf.expand_dims(Q, axis=-1), tf.expand_dims(K, axis=-1), transpose_b=True)

    f = layers.Softmax()(f)
    
#     y = tf.squeeze(tf.matmul(f, tf.expand_dims(V, axis=-1)))
    y = tf.matmul(f, tf.expand_dims(V, axis=-1))
#     y = tf.reshape(y, (-1, hx.shape[1], hx.shape[2], y.shape[-1]))

    z = tf.math.add(x_in, layers.Dense(hx.shape[-1])(tf.squeeze(y, axis=-1)))

    return z

def swish(x):
    return tf.keras.activations.swish(x)

def insnorm(x):
    return tfa.layers.InstanceNormalization(axis=-1)(x)


def pre_act_resblk(x, act_func, norm_func, kernel_size=(3, 3), strides=(1, 1), padding="valid", temb=None):
    h = ReflectionPadding2D()(x)
    h = act_func(norm_func(h))
    h = layers.Conv2D(x.shape[-1], kernel_size,
                                    strides=strides, padding=padding)(h)
    if not temb == None:
        temb = act_func(temb)
        h +=  tf.reshape(layers.Dense(h.shape[-1])(temb), (-1, 1, 1,h.shape[-1]))
        
    h = ReflectionPadding2D()(h)
    h = act_func(norm_func(h))
    h = layers.Conv2D(x.shape[-1], kernel_size,
                                    strides=strides, padding=padding)(h)
    
    return x+h


def downsample(x, filters, kernel_size=(3, 3), padding="same", factor=2):
    return layers.Conv2D(filters, kernel_size,
                                    strides=(factor, factor), padding=padding)(x)

def upsample(x, filters, kernel_size=(3, 3), padding="same", factor=2):
    return layers.Conv2DTranspose(filters, kernel_size,
                                    strides=(factor, factor), padding=padding)(x)

In [ ]:
input_size=(136,)

In [ ]:
def get_classifier1(
    act_func, 
    filters=64,
    num_attn_blocks=1
):
    lnd_input = layers.Input(shape=input_size)
    x = layers.Dense(filters)(lnd_input)

    x = act_func(x)

    # Attention blocks
    for _ in range(num_attn_blocks):

        x = NonlocalGaussian_1D(x)

    x = layers.Dense(4)(x)

    model = Model(inputs=lnd_input, outputs=x)
    return model

def get_classifier2(
    act_func, 
    filters=64
):
    lnd_input = layers.Input(shape=input_size)
    x = layers.Dense(filters)(lnd_input)

    x = act_func(x)

#     # Attention blocks
#     for _ in range(num_attn_blocks):

#         x = NonlocalGaussian_1D(x)

    x = layers.Dense(4)(x)

    model = Model(inputs=lnd_input, outputs=x)
    return model


In [ ]:
classi_F = get_classifier1(swish,filters=512)

In [ ]:
tf.keras.utils.plot_model(classi_F, to_file="./model_plots/model.png", show_shapes=True, show_layer_activations=True)

In [ ]:
classi_F.summary()

In [ ]:
# def BiCE(d_real, d_fake):
#         d_real = tf.math.sigmoid(d_real)
#         d_fake = tf.math.sigmoid(d_fake)
#         return tf.nn.compute_average_loss(tf.reduce_mean((-d_real*tf.math.log(tf.clip_by_value(d_fake,
#                             1e-16, 1)) - (1.0 - d_real)*tf.math.log(
#                             tf.clip_by_value(1.0 - d_fake,
#                             1e-16, 1))),
#                             axis=-1))


cce = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True)

def L2(pred, y):
        return tf.reduce_mean(tf.norm(pred-y, axis=-1))

In [ ]:
a = tf.random.normal((64,68))
b = tf.random.normal((64,68))

L2(a, b)



In [ ]:
class ValaroModel1(Model):
    def __init__(
        self,
        classifer_F,
    ):
        super().__init__()
        self.classifer_F = classifer_F

    def compile(
        self,
        classifer_F_optimizer,
        classifier_loss_fn,
    ):
        super().compile()
        self.classifer_F_optimizer = classifer_F_optimizer
        self.classifier_loss_fn = classifier_loss_fn


    def train_step(self, batch_data):
        lnd, valaro = batch_data

        with tf.GradientTape(persistent=True) as tape:

            pred = self.classifer_F(lnd, training=True)

            loss = self.classifier_loss_fn(valaro, pred)

        # Get the gradients for the classifer
        grads_F = tape.gradient(loss, self.classifer_F.trainable_variables)

        # Update the weights of the classifer
        self.classifer_F_optimizer.apply_gradients(
            zip(grads_F, self.classifer_F.trainable_variables)
        )

        return {
            "F_loss": loss,
                }

In [ ]:
# Create classifier model
valaro1 = ValaroModel1(
    classifer_F=classi_F
)
mse = tf.keras.losses.MeanSquaredError()
# Compile the model
valaro1.compile(
    classifer_F_optimizer=tf.keras.optimizers.SGD(learning_rate=2e-4, momentum=0.0),
    classifier_loss_fn=mse,
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# !pip install scikit-learn

In [ ]:
from sklearn.metrics import r2_score

class AccuracyMonitor(tf.keras.callbacks.Callback):
    """A callback to generate and save images after each epoch"""

    def __init__(self, test_set):
        self.ds_test = test_set

    def on_epoch_end(self, epoch, logs=None):
#         print(self.model.num_samples_used)
        l2s = []
        r2s = []
        for i, data in enumerate(self.ds_test):
            
            lnds, valaros = data
            pred = self.model.classifer_F(lnds)

            l2s.append(tf.reduce_mean(tf.norm(pred-valaros, axis=-1)))
            r2s.append(tf.constant(r2_score(pred.numpy(),valaros.numpy())))
            
        print("\n Avg. L2:", tf.reduce_mean(l2s))
        print(" Avg. R2:", tf.reduce_mean(r2s))

In [ ]:
ds.cardinality()

In [ ]:
train_size = int(0.8 * ds.cardinality().numpy())
# test_size = int(0.2 * ds.cardinality)

train_ds = ds.take(train_size)
test_ds = ds.skip(train_size)

accmon = AccuracyMonitor(test_ds)

valaro1.fit(
    train_ds,
    epochs=100,
    callbacks=[accmon, tensorboard_callback],
)


In [ ]:
x = tf.random.normal((1,136))
pred = valaro1.classifer_F(x)

y = tf.constant([[1.,1.]])
y, pred

# from sklearn.metrics import r2_score

# r2_score(pred.numpy(),y.numpy())

In [ ]:
tf.norm(pred-y, axis=-1)